## DATA VALIDATION
_Vibha Hodachalli_

This notebook details the process of validating data from two data marts, one in Oracle and the other in Snowflake. 


## 1. Get subset from Snowflake and Oracle SQL Dev  

- The primary key can be used to sort the data.  
     - primary key is specified in the columns tab of Oracle SQL Dev when a table is selected.

In [ ]:
//Role must be set to access the marts
USE ROLE ADMIN;

//It may be a good idea to skim over the data and see what subset of it you want to use
SELECT * FROM SNOWFLAKE_MART ORDER BY PRIMARY_KEY ASC;

//I observed that the DATE_COL and ... dates range from 2010 to 2024, I decided to take a subset where the ... was b/w 2020-2021. Ordered by PRIMARY_KEY (which is the primary key) ascending
SELECT * FROM SNOWFLAKE_MART WHERE YEAR(DATE_COL) BETWEEN 2020 AND 2021 ORDER BY PRIMARY_KEY ASC;

In [ ]:
//SQL query to get subset from Oracle SQL Dev
SELECT 
COL1,
TO_CHAR(DATE_COL, 'YYYY-MM-DD HH24:MI:SS".000 Z"') AS COL2,
COL3,
PRIMARY_KEY
FROM ORACLE_MART WHERE EXTRACT(YEAR FROM DATE_COL) BETWEEN 2020 AND 2021 ORDER BY PRIMARY_KEY ASC;

## 2. Create Tables

- You can create tables in Snowflake from files by clicking the + button in the far left upper corner

- From the two queries above, I exported the results to csv files and created two tables in VIBHA_TEST.TEST  
1. ORACLE_TABLE from the oracle query
2. SNOWFLAKE_TABLE from the snowflake query

- Make sure that the data types for each column are the same when creating the tables.  
    - _You can just change all data types to VARCHAR for simplicity_

## 3. Observe and Update Tables  
Query the tables and look over the data.   
Make any updates if necessary.

_For example:_  
- I observed that some columns in the ORACLE_TABLE table had empty values where the SNOWFLAKE_TABLE table had NULL values.


In [ ]:
USE ROLE VIBHA;

SELECT * FROM VIBHA.SNOWFLAKE_TABLE WHERE COL1 IS NULL; //This query produced results

SELECT * FROM VIBHA.ORACLE_TABLE WHERE COL1 IS NULL; //This query did not

SELECT * FROM VIBHA.ORACLE_TABLE WHERE COL1 = ''; //Here I realized that the "null" values were being displayed as empty strings

//To fix this issue, I used the following queries to update the table values in VIBHA.ORACLE_TABLE:
UPDATE VIBHA.ORACLE_TABLE
SET COL1 = CASE WHEN COL1 = '' THEN NULL ELSE COL1 END;

## 4. Run below Python script using appropriate query in lines 8 and 9
- If any differences are found, you will be prompted with a button to download a csv of the rows that differ

In [ ]:
import pandas as pd
from snowflake.snowpark.context import get_active_session
import streamlit as st

# Get active Snowflake session
session = get_active_session()

# Fetch data from Snowflake and Oracle into dataframes
oracle_df = session.sql('SELECT * FROM VIBHA.ORACLE_TABLE LIMIT 10000')
snowflake_df = session.sql('SELECT * FROM VIBHA.SNOWFLAKE_TABLE LIMIT 10000')

print("oracle_df: ")
oracle_df.show()

print("snowflake_df: ")
snowflake_df.show()

# Convert Snowflake and Oracle dataframes to pandas dataframes
snowflake_pandas_df = snowflake_df.to_pandas()
oracle_pandas_df = oracle_df.to_pandas()

# Add a new column "Table" whose value is Oracle/Snowflake respectively
oracle_pandas_df.insert(loc=0, column='Table', value='Oracle')
snowflake_pandas_df.insert(loc=0, column='Table', value='Snowflake')

# Compare dataframes excluding the 'Table' column
diff_df = oracle_pandas_df.loc[:, oracle_pandas_df.columns != 'Table'].compare(snowflake_pandas_df.loc[:, snowflake_pandas_df.columns != 'Table'], align_axis=0, keep_equal=False)
print("Results from comparison:\n",diff_df)

# Get indices where there are differences
diff_indices = diff_df.index.get_level_values(0).unique()

# Initialize an empty list to store dataframes to concatenate
concat_dataframes = []

# Loop through the different indices
for idx in diff_indices:
    # Append the row from Oracle dataframe to concat_dataframes list
    concat_dataframes.append(oracle_pandas_df.iloc[[idx]])

    # Append the row from Snowflake dataframe to concat_dataframes list
    concat_dataframes.append(snowflake_pandas_df.iloc[[idx]])
if concat_dataframes:
   # Concatenate all dataframes in concat_dataframes list into result_df
   result_df = pd.concat(concat_dataframes, ignore_index=True)

    # Now result_df contains all rows from Oracle and Snowflake where differences were found
   print("All column and row values for the differences:\n",result_df)
    
   #Download the dataframe as a .csv file
   @st.cache_data
   def convert_df(df):
    return df.to_csv(index=False).encode('utf-8')


   csv = convert_df(result_df)

   st.download_button(
   "Press to Download CSV",
   csv,
   "file.csv",
   "text/csv",
   key='download-csv'
   )
else:
    print("No differences found!")

The results from the above comparison should return an empty data frame, meaning that no differences were found!

The data can always be further validated though ->

## 5. Analyze and Validate Further
- If differences are found,
    - Analyze sources of differences by looking at the downloadable .csv file, SQL stored procedures used to create the marts, and source data in cerner millenium
- Validate the next set of rows to see if any new differences come up

In [ ]:
// In the first run of the script, 10,000 rows each from the Oracle and Snowflake marts were compared
// Next we might compare the following 10,000 rows
// In order to be compare the next 10,000 and not repeat any rows from the initial validation, the following query can be used:
SELECT * 
FROM VIBHA.ORACLE_TABLE 
WHERE PRIMARY_KEY 
// The following conditional is the original query used in the initial validation run
NOT IN (SELECT PRIMARY_KEY FROM VIBHA.ORACLE_TABLE ORDER BY PRIMARY_KEY ASC LIMIT 10000) ORDER BY PRIMARY_KEY ASC LIMIT 10000;

SELECT * 
FROM VIBHA.SNOWFLAKE_TABLE 
WHERE PRIMARY_KEY 
// The following conditional is the original query used in the initial validation run
NOT IN (SELECT PRIMARY_KEY FROM VIBHA.SNOWFLAKE_TABLE  ORDER BY PRIMARY_KEY ASC LIMIT 10000) ORDER BY PRIMARY_KEY ASC LIMIT 10000;

## Further Validation (cont)
- The same Python script as used in the initial validation can be used again
    - Replace the query in lines 8 and 9 with the query developed above as to not repeat the validation of any rows

In [ ]:
import pandas as pd
from snowflake.snowpark.context import get_active_session
import streamlit as st

# Get active Snowflake session
session = get_active_session()

# Fetch data from Snowflake and Oracle into dataframes
# session.sql('YOUR QUERY GOES HERE')
oracle_df = session.sql('SELECT * FROM VIBHA.ORACLE_TABLE  WHERE PRIMARY_KEY NOT IN (SELECT PRIMARY_KEY FROM VIBHA.ORACLE_TABLE ORDER BY PRIMARY_KEY ASC LIMIT 10000) ORDER BY PRIMARY_KEY ASC LIMIT 10000')
snowflake_df = session.sql('SELECT * FROM VIBHA.SNOWFLAKE_TABLE WHERE PRIMARY_KEY NOT IN (SELECT PRIMARY_KEY FROM VIBHA.SNOWFLAKE_TABLE ORDER BY PRIMARY_KEY ASC LIMIT 10000) ORDER BY PRIMARY_KEY ASC LIMIT 10000')

print("oracle_df: ")
oracle_df.show()

print("snowflake_df: ")
snowflake_df.show()

# Convert Snowflake and Oracle dataframes to pandas dataframes
snowflake_pandas_df = snowflake_df.to_pandas()
oracle_pandas_df = oracle_df.to_pandas()

# Add a new column "Table" whose value is Oracle/Snowflake respectively
oracle_pandas_df.insert(loc=0, column='Table', value='Oracle')
snowflake_pandas_df.insert(loc=0, column='Table', value='Snowflake')

# Compare dataframes excluding the 'Table' column
diff_df = oracle_pandas_df.loc[:, oracle_pandas_df.columns != 'Table'].compare(snowflake_pandas_df.loc[:, snowflake_pandas_df.columns != 'Table'], align_axis=0, keep_equal=True)
print("Results from comparison:\n",diff_df)

# Get indices where there are differences
diff_indices = diff_df.index.get_level_values(0).unique()

# Initialize an empty list to store dataframes to concatenate
concat_dataframes = []

# Loop through the different indices
for idx in diff_indices:
    # Append the row from Oracle dataframe to concat_dataframes list
    concat_dataframes.append(oracle_pandas_df.iloc[[idx]])

    # Append the row from Snowflake dataframe to concat_dataframes list
    concat_dataframes.append(snowflake_pandas_df.iloc[[idx]])
if concat_dataframes:
   # Concatenate all dataframes in concat_dataframes list into result_df
   result_df = pd.concat(concat_dataframes, ignore_index=True)

    # Now result_df contains all rows from Oracle and Snowflake where differences were found
   print("All column and row values for the differences:\n",result_df)
   
    #Download the dataframe as a .csv file
   @st.cache_data
   def convert_df(df):
    return df.to_csv(index=False).encode('utf-8')


   csv = convert_df(result_df)

   st.download_button(
   "Press to Download CSV",
   csv,
   "file.csv",
   "text/csv",
   key='download-csv'
   )
else:
    print("No differences found!")

## That's all!
You can always validate even further and keep repeating step 5, while updating the query used :)